# GEOJSON Points

    Create Lon Lat points from City, State names
    Imports CSV File with city names
    Uses geoapify to get lon lat

In [1]:
# Dependencies and Setup
import pandas as pd
import requests as req
import time
from geojson import Point, Feature, FeatureCollection, dump

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the UAP CSV file into a Pandas DataFrame
uap_data_df = pd.read_csv("uap_data_output.csv")

# Group city, state, country to limit geoapify queries
cities = uap_data_df.groupby(['City','State','Country']).sum().reset_index()

# Save queries into a list
geoapify_queries = []
geoapify_query = 'https://api.geoapify.com/v1/geocode/search?text='
for colname, data in cities.iterrows():
    city = data.iloc[0]
    State = data.iloc[1]
    Country = data.iloc[2]
    geoapify_queries.append(geoapify_query + 
                            city + ',' + 
                            State + ',' + 
                            Country + '&format=json&apiKey=' + geoapify_key)

uap_data_df.sample()

,Occurred,Lon,Lat,City,State,Country,Shape,Summary,Media,Explanation
71,4/19/1966,-122.674194,45.520247,Portland,OR,USA,Formation,I was looking northeast and saw 4 very bright ...,NaN,NaN


In [4]:
# limit the for loop as a safety measure
loop = 0

# Finding the lat lon for the city locations
# Adding / updating lat lon in the uap dataframe
for geo in geoapify_queries:

    try:
        # req geoapify
        city_json = req.get(geo).json()

        lon = city_json['results'][0]['lon']
        lat = city_json['results'][0]['lat']
        city_name = city_json['results'][0]['city']
        state_abbr = city_json['results'][0]['state_code']

        # add lon lat to dataframe
        uap_data_df.loc[(uap_data_df['City'] == city_name) & (uap_data_df['State'] == state_abbr), 'Lon'] = lon
        uap_data_df.loc[(uap_data_df['City'] == city_name) & (uap_data_df['State'] == state_abbr), 'Lat'] = lat
        
    except:
        print(city_name)
    
    if loop == 101:
        break
    loop += 1

    time.sleep(1)

uap_data_df.sample()

,Occurred,Lon,Lat,City,State,Country,Shape,Summary,Media,Explanation
29,10/1/2005,-81.482812,41.136273,Cuyahoga Falls,OH,USA,Other,"V-shaped object seen in sky of Cuyahoga Falls,...",NaN,NaN


In [5]:
# Save the data to a csv file
uap_data_df.to_csv('uap_data_output.csv', index=None)

In [6]:
# may want to check the csv file for completeness before running this
# Output the geojson data
feature_collections = {}

count = len(uap_data_df)
features = []
for i in range(count):
    testout = uap_data_df.iloc[i]
    point = Point((testout.loc[('Lon')], testout.loc[('Lat')]))

    features.append(Feature(geometry=point, 
                            properties={
                                "Occurred": testout.loc[('Occurred')],
                                "City": testout.loc[('City')],
                                "Country": testout.loc[('Country')],
                                "Shape": testout.loc[('Shape')],
                                "Summary": testout.loc[('Summary')]
                                }))

feature_collection = FeatureCollection(features)


with open('uap_data_output.geojson', 'w') as f:
    dump(feature_collection, f)
